In [212]:
%%capture
%run 04_account.ipynb
%run 05_block.ipynb

## Miner

Mining is a crucial component on any blockchain relying on proof of work (abbr. pow). It is used to reach consensus in a network of anonymous decentralized nodes. Anyone with an account `acc` can become a miner. 

The miner takes a new block and tries to solve a specific computational puzzle, a specific hash. This can only be done by brute force. 

`coinbase` creates a tx to the miner as a reward for providing the pow (also called coinbase tx). 

In [213]:
class Miner:
    def __init__(self, acc): 
        self.acc = acc
        self.pub = acc.pub
        
    def mine(self, txs, prev_header, diff, reward, attempts=1000):
        txs.insert(0, self.coinbase(reward))
        mt = MerkleTree(txs)
        bh = Header(mt.root, prev_header.hash, prev_header.number+1, len(txs))
        bh.diff   = diff
        bh.reward = reward
        bh.miner  = self.pub
        bh.mined  = True
        bh_b = bytes(bh)
        nonce = 0
        for i in range(attempts):
            candidate   = bh_b + str(nonce).encode()
            candidate_h = sha(candidate)
            if candidate_h[2:2+diff] == '0'*diff: break
            nonce += 1
            assert nonce != attempts, 'no nonce could be found'
        bh.nonce  = nonce
        return Block(bh, txs)
        
    def coinbase(self, reward): 
        return self.acc.sign(TX(self.pub, self.pub, reward, 0, self.acc.nonce))

### Genesis Block

Is the first block in the blockchain. This is how it all begins. 

In [214]:
def mine_genesis(txs):
    mt  = MerkleTree(txs)
    bh  = Header(mt.root, rh(), 0, len(txs))
    return Block(bh, txs)

gb  = mine_genesis(r_stxs(2)); print(gb)

root:          💹 0x5308a8e7f4...106
hash:          📆 0x60a49e7932...82d
prev_hash:     👨 0x027b65149a...845
number:        0
n_txs:         2
mined:         False
time:          Tue Mar 30 22:04:30 2021
volume:        15.0 ether
fees:          0.09 ether

txs:
📅 0x5f22c1925b...74f
👩 0x03858325e3...853


### Mining

Mine a block on top of the genesis block

In [215]:
miner = Miner(acc1)
mb = miner.mine(r_stxs(10), gb.bh, 1, 100); print(mb)

root:          📡 0x7bcc363ad1...49d
hash:          🕉 0xe3514dfacd...815
prev_hash:     📆 0x60a49e7932...82d
number:        1
n_txs:         11
mined:         True
time:          Tue Mar 30 22:04:31 2021
diff:          1
reward:        100
miner:         🕣 0xfdEbA52fDa...66b
nonce:         17
volume:        152.0 ether
fees:          0.52 ether

txs:
📛 0x75ffef745d...3d0
💋 0x25affa7a01...4b8
💏 0x2950dd949b...ba9
🔽 0xd719257beb...68c
🕗 0xf1d552ea04...527
📧 0x81522997fb...348
🔋 0xa57013946c...202
📞 0x78b7e06a70...6c6
👶 0x102ff0ad6d...e59
💊 0x24ebac6d18...b00
🕡 0xfbfbb6d320...633


### PoW validation

We can validate a mined blocks pow very easily.

In [208]:
def val_pow(mb):
    bh          = mb.bh
    mb_b        = bytes(bh)
    candidate   = mb_b + str(bh.nonce).encode()
    candidate_h = sha(candidate)
    if candidate_h[2:2+bh.diff] == '0'*bh.diff: return True
    else                                      : return False
    
assert val_pow(mb)

If we change something in the block the pow should break.

In [210]:
mb.bh.prev_hash = rh()
assert not val_pow(mb)